<h1>Generate and Improve Image Descriptions</h1>

In this notebook we prompt a model to describe our images and then we ask the model to improve descriptions. (a-few-shots prompt)

This involves using an image as a scenario of survey questions, and piping the answer to a question into another question.

In [1]:
from edsl import QuestionFreeText

q1 = QuestionFreeText(
    question_name = "describe",
    question_text = """
    I have an image of an object that I would like you to describe in detail. 
    Based on the image provided, please write a description that includes:

    1. The object's appearance, including color, shape, and material.
    2. Any notable design features or patterns.
    3. The object's functionality or potential use.
    4. The overall impression or aesthetic of the object.
    
    Here is the image: {{ image }}. Please make your description as precise and vivid as possible.
    """
)

In [2]:
from edsl import QuestionFreeText

q2 = QuestionFreeText(
    question_name = "improvements",
    question_text = """
    Evaluate the description of this image on a scale from 1 to 10.
    Explain the reasoning behind your evaluation.
    Image: {{ image }}
    Description: {{ describe.answer }}
    """
)

In [3]:
IMAGE_UUIDS = {
    'cabin_luggage': ['4f41ad44-6472-4403-8e59-0fb0e5eca575', 'e30bbefa-b135-4a46-bf32-ed9437be8733', 'a56079a4-1f50-4019-9344-947101bd3e4b'],
    'packing_cubes': ['7c046f64-1209-447d-a68a-b984f3508e0b', '82ffd7a6-87dc-4365-9c9f-2e76263908be', '851dedbe-3b32-45ca-8c32-4ddc510f7327'],
    'water_bottle': ['2a81aec9-c164-4d29-9281-360547f04ac4', 'd34d3ec1-073d-45b9-baca-a0d3411e9e0c', 'ff0ab897-50b8-458e-bfa5-7550b8a48648']
}

In [5]:
%%time

from edsl import Scenario, ScenarioList, FileStore

def prefetch_images(image_uuids):
    return {
        product: {
            index: FileStore.pull(uuid) for index, uuid in enumerate(uuids)
        }
        for product, uuids in image_uuids.items()
    }

pre_fetched_images = prefetch_images(IMAGE_UUIDS)

sl = ScenarioList([
    Scenario({
        "question_name": f"{product}_{i + 1}",
        "image": pre_fetched_images[product][i]
    })
    for product in IMAGE_UUIDS
    for i in range(3)
])

sl

CPU times: user 315 ms, sys: 294 ms, total: 610 ms
Wall time: 19.9 s


,question_name,image
0,cabin_luggage_1,FileStore: self.path
1,cabin_luggage_2,FileStore: self.path
2,cabin_luggage_3,FileStore: self.path
3,packing_cubes_1,FileStore: self.path
4,packing_cubes_2,FileStore: self.path
5,packing_cubes_3,FileStore: self.path
6,water_bottle_1,FileStore: self.path
7,water_bottle_2,FileStore: self.path
8,water_bottle_3,FileStore: self.path


In [6]:
%%time

from edsl import Survey

survey = Survey(questions = [q1, q2])

CPU times: user 173 μs, sys: 7 μs, total: 180 μs
Wall time: 182 μs


In [7]:
from edsl import Model

Model.services()

,Service Name
0,anthropic
1,azure
2,bedrock
3,deep_infra
4,deepseek
5,google
6,groq
7,mistral
8,ollama
9,openai


In [8]:
%%time

from edsl import Model, ModelList

ml = ModelList([
  Model("gpt-4o", service_name = "openai", temperature = 0, max_tokens = 5000),
  Model("gemini-1.5-flash", service_name = "google", temperature = 0, maxOutputTokens = 5000)
])

ml

CPU times: user 1.51 ms, sys: 1.86 ms, total: 3.37 ms
Wall time: 2.07 ms


,topK,top_logprobs,presence_penalty,frequency_penalty,temperature,top_p,logprobs,topP,maxOutputTokens,model,stopSequences,max_tokens,inference_service
0,nan,3.000000,0.000000,0.000000,0,1.000000,False,nan,nan,gpt-4o,nan,5000.000000,openai
1,1.000000,nan,nan,nan,0,nan,nan,1.000000,5000.000000,gemini-1.5-flash,[],nan,google


In [16]:
# import os
from dotenv import load_dotenv

load_dotenv()  # By default, loads from .env in the current directory

# Test if the API keys are loaded
# print(os.getenv("OPENAI_API_KEY"))
# print(os.getenv("GOOGLE_API_KEY"))
# print(os.getenv("EXPECTED_PARROT_API_KEY"))


True

In [13]:
%%time

results = survey.by(sl).by(ml).run()

CPU times: user 1.99 s, sys: 279 ms, total: 2.27 s
Wall time: 25.5 s


In [14]:
results.select('model', 'scenario.question_name', 'answer.*')

,model.model,scenario.question_name,answer.improvements,answer.describe
0,gpt-4o,cabin_luggage_1,"I would rate the description a 9 out of 10. **Reasoning:** 1. **Accuracy**: The description accurately captures the key features of the suitcase, such as its metallic silver color, rectangular shape with rounded edges, and vertical grooves. These details are clearly visible in the image. 2. **Detail**: It provides a comprehensive overview, covering appearance, design features, and functionality. The mention of reinforced corners, multi-directional wheels, and a retractable handle adds depth to the description. 3. **Clarity**: The language is clear and easy to understand, making it accessible to a wide audience. 4. **Overall Impression**: The description effectively conveys the luxury and durability of the suitcase, aligning with the visual impression from the image. 5. **Minor Improvement**: While the description is thorough, it could include more specific details about the interior features or dimensions for a perfect score. Overall, the description is well-crafted and aligns closely with the visual elements of the image.","The object in the image is a sleek, modern suitcase. Here's a detailed description: 1. **Appearance**: - **Color**: The suitcase is a metallic silver, giving it a polished and sophisticated look. - **Shape**: It has a rectangular shape with rounded edges, providing a streamlined appearance. - **Material**: The suitcase appears to be made of a durable, hard-shell material, likely aluminum, known for its strength and lightweight properties. 2. **Design Features**: - The suitcase features vertical grooves running along its body, adding texture and enhancing its structural integrity. - The corners are reinforced with additional metal plating, providing extra protection against impacts. - It has a sturdy handle on top, likely retractable, for easy carrying. - The wheels are multi-directional, allowing for smooth and effortless movement. 3. **Functionality**: - This suitcase is designed for travel, offering ample space for packing personal belongings. - The hard-shell exterior provides excellent protection for the contents inside. - The wheels and handle make it highly portable, ideal for navigating through airports or city streets. 4. **Overall Impression**: - The suitcase exudes a sense of luxury and durability, combining functionality with a high-end aesthetic. - Its minimalist design and metallic finish give it a timeless and professional look, appealing to frequent travelers and business professionals alike."
1,gemini-1.5-flash,cabin_luggage_1,"I would rate the description of the Rimowa Essential Cabin suitcase a 9 out of 10. **Reasons for the high score:** * **Accuracy:** The description is highly accurate in its depiction of the suitcase's appearance, design features, and functionality. It correctly identifies the material (aluminum), color (light brushed silver), and key design elements like the grooved pattern, recessed nameplate, and wheels. The description of the aesthetic as ""modern, understated elegance"" is also spot on. * **Detail:** The level of detail is excellent. The writer doesn't just state facts; they describe the *texture* of the aluminum, the *subtlety* of the grooves and logo, and the overall *feeling* of the suitcase. This creates a vivid image in the reader's mind. * **Structure:** The description is well-structured, logically progressing from appearance to design features to functionality and finally to aesthetic. This makes it easy to follow and understand. * **Vocabulary:** The vocabulary is sophisticated and appropriate for the subject matter. Words like ""minimalist,"" ""recessed,"" ""embossed,"" and ""ostentation"" elevate the description beyond a simple listing of features. **Reason for not giving a perfect 10:** * **Minor Omission:** While the description is thorough, it could benefit from mentioning the handle's extendable nature (if it is extendable) and the presence of any TSA-